# PGEN(Prompt-based domain text Generation)
사전 학습된 언어 모델(GPT-2 등)을 활용하여 도메인 내 텍스트 데이터를 대규모로 생성하는 방법
- [고려대학교 구름](https://github.com/nlpai-lab/KULLM)
- 참고 https://wikidocs.net/226090

## 0. 라이브러리

In [44]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

In [45]:
import numpy as np
import pandas as pd
#from torch.utils.data import Dataset, DataLoader
#
#import torch.optim as optim

In [46]:
!pip install utils

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 1. 데이터 불러오기

In [47]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/StoneMaenggu/coda-modeling/main/Gloss2Text/Dataset/NIASL2020_LLM.csv')
df = df[df.g_type=='SEN']
df = df[['start_end_time', 'gloss']]
df['gloss'] = df['gloss'].astype(str).apply(lambda x: x.replace('|', ' '))
df.text = ''
df

,start_end_time,gloss
48000,"[[1.975, 2.856], [2.989, 3.489], [3.608, 4.203...",기차 곳 내리다 맞다
48001,"[[1.626, 2.389], [2.477, 2.977], [3.09, 3.59],...",편지 저기 곳 다음
48002,"[[2.362, 3.974]]",위험
48003,"[[1.856, 2.638], [2.799, 3.388], [3.557, 4.186...",스타벅스 곳 저기 전 도착 차내리다
48004,"[[1.213, 1.899], [2.107, 2.607], [2.843, 3.343...",여기 송파 지하철 맞다
...,...,...
79995,"[[2.028, 2.528], [2.647, 3.483]]",그남자 맞다
79996,"[[1.959, 2.668], [2.782, 3.282], [3.42, 4.676]]",손바닥찍다 가다 내리다
79997,"[[1.847, 2.096], [2.41, 2.774], [3.095, 4.134]]",핸드폰 잃어버리다 찾다
79998,"[[1.407, 2.179], [2.37, 3.278]]",가방 실종


## 2. LLM 불러오기

### git clone

In [48]:
#!sudo apt update
#!sudo apt install git
#!git --version

In [49]:
!git clone -b kullm_v2 https://github.com/nlpai-lab/KULLM.git

fatal: 대상 경로가('KULLM') 이미 있고 빈 디렉터리가 아닙니다.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
!cd KULLM

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [51]:
#!conda create -n KULLM python=3.10
!pip install -U torch transformers tokenizers accelerate jupyter

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached tokenizers-0.19.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.0-py3-none-any.whl (9.5 MB)
Using cached tokenizers-0.19.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [52]:
!pip install torch transformers==4.38.2 accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


In [53]:
!pip install utils

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
!pwd

/home/stonemaeng/g2t/Dataset


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [55]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

import sys
sys.path.append('./KULLM/utils')
from prompter import Prompter

In [56]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')

In [57]:
MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,).to(device)

model.eval()

/home/stonemaeng/anaconda3/envs/g2t/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 15.70 GiB of which 3.50 MiB is free. Including non-PyTorch memory, this process has 15.68 GiB memory in use. Of the allocated memory 15.40 GiB is allocated by PyTorch, and 10.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### model

In [58]:
pipe = pipeline("text-generation", model=model, tokenizer=MODEL, device=0)
prompter = Prompter("/home/stonemaeng/g2t/Dataset/KULLM/templates/kullm")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [59]:
def infer(instruction=instruction, input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(prompt, max_length=512, temperature=0.2, num_beams=5, eos_token_id=2)
    s = output[0]["generated_text"]
    result = prompter.get_response(s)
    return result

In [41]:
instruction = '''태스크 정의: 단어 집합의 형태로 입력을 받으면 이를 완전한 문장으로 완성하는 태스크이다. 이때 입력 단어에 없는 정보는 출력에 포함되면 안된다.
올바른 예시:
입력: 어디 와 은행 나 가다
출력: 어디인가요? 나는 은행에 갑니다.
잘못된 예시:
입력: 나가다 지금
출력: 지금 학교에 가고 있다.
아래 문장에 대해서 위 태스크를 수행하고 결과만 반환해줘.'''

In [64]:
infer(input_text="일 해결 좋다 기분")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 34.00 MiB. GPU 0 has a total capacity of 15.70 GiB of which 11.50 MiB is free. Including non-PyTorch memory, this process has 15.67 GiB memory in use. Of the allocated memory 15.00 GiB is allocated by PyTorch, and 395.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [62]:
del model
import gc
gc.collect()
torch.cuda.empty_cache()
